In [ ]:
import pyarrow.dataset as ds
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.fs as fs

In [ ]:
# Crear bucket manualmente antes (usando boto3 o minio client)
import boto3
s3 = boto3.client('s3', 
                 endpoint_url='http://minio:9000',
                 aws_access_key_id='admin',
                 aws_secret_access_key='password')
s3.create_bucket(Bucket='mi-bucket-iceberg')

In [ ]:
minio = fs.S3FileSystem(
    access_key="admin",
    secret_key="password",
    endpoint_override="http://minio:9000",
)

In [ ]:
path = ""

In [ ]:
dataset_3 = ds.dataset(path, filesystem=minio, format="parquet")

In [ ]:
table_3 = dataset_3.to_table()
print(table_3.schema)

In [ ]:
for file_info in minio.get_file_info([path, "taxis/taxis_parquet/df_data"]):
    print(file_info)


In [ ]:
arrow_schema = table_3.schema
print(arrow_schema)

In [ ]:
from pyiceberg.catalog import load_catalog

catalog = load_catalog(
    "nessie",
    **{
        "uri": "http://nessie:19120/iceberg/main",
    }
)


In [ ]:
namespaces = catalog.list_namespaces()
print(namespaces)


In [ ]:
catalog.create_namespace("taxis")

namespaces = catalog.list_namespaces()
print(namespaces)

In [ ]:
catalog.create_table(
    "taxis.taxis_iceberg", 
    arrow_schema,
    properties={
        'warehouse': 's3://mi-bucket-iceberg/',
        's3.endpoint': 'http://minio:9000',
        's3.access-key-id': 'admin',
        's3.secret-access-key': 'password'
    }
)

In [ ]:
taxis_iceberg = catalog.load_table("taxis.taxis_iceberg")
taxis_iceberg.append(table_3)